In [12]:
import networkx as nx
import duckdb

import pandas as pd
import matplotlib.pyplot as plt
from pycelonis import get_celonis

url = "https://louis-herrmann-rwth-aachen-de.training.celonis.cloud"
api = "NWE2NjdjOGEtYTkyMS00NDYyLTk0M2EtZjFiYjdhZDA5MTYzOmZJSDIydFd3TEwrQkUwV2tBVkhtN0N5VFI1aHdWYVJ2TDJVUWpoL2U5cUE4"

celonis = get_celonis(url, api)

ERROR:pycelonis:Couldn't connect to IBC, trying to switch key type...


2022-01-19 16:11:40 - pycelonis: Login successful! Hello Louis. PyCelonis Version: 1.5.14
2022-01-19 16:11:40 - pycelonis: Your key has following permissions:
[
    {
        "permissions": [
            "EDIT_ALL_SPACES",
            "MANAGE_PERMISSIONS",
            "CREATE_SPACE",
            "DELETE_ALL_SPACES"
        ],
        "serviceName": "package-manager"
    },
    {
        "permissions": [
            "EDIT_AGENTS",
            "VIEW_AGENTS",
            "REGISTER_AGENTS",
            "MANAGE_PERMISSIONS"
        ],
        "serviceName": "workflows"
    },
    {
        "permissions": [
            "EDIT_CLIENT_SETTINGS",
            "EDIT_USERS"
        ],
        "serviceName": "task-mining"
    },
    {
        "permissions": [
            "MANAGE_SSO_SETTINGS",
            "USE_AUDIT_LOGS_API",
            "MANAGE_GENERAL_SETTINGS",
            "MANAGE_GROUPS",
            "MANAGE_APPLICATIONS",
            "MANAGE_LICENSE_SETTINGS",
            "MANAGE_MEMBERS",
   

In [13]:
data_pool = celonis.pools.find("OCEL_Pool1")
data_model = data_pool.datamodels.find("OCEL_Model1")

In [14]:
class Table:
    # should data model be a celonis data model or own class?
    def __init__(self, name, data_model, data=None):
        self.name = name
        self.data_model = data_model
        self.foreignKeys = {"as_source": [], "as_target": []}
        for dictionary in data_model.foreign_keys.find_keys_by_source_name(name):
            self.foreignKeys["as_source"].append(dictionary)
        for dictionary in data_model.foreign_keys.find_keys_by_target_name(name):
            self.foreignKeys["as_target"].append(dictionary)
        
        self.data = data
    
    def fetchData(self):
        if self.data is None:
            self.data = data_model.tables.find(self.name).get_data_frame()
    
    def getData(self):
        if self.data is None:
            self.fetchData()
        return self.data

In [15]:
class ActivityTable(Table):
    
    # also make constructor with more parameters (e.g. passing columns...)
    
    def __init__(self, *args): # args: [name, data_model, data, actColumn, caseColumn, timestampColumn]
        if len(args) < 2:
            print("ActivityTable constructor needs more arguments")
        name = args[0]
        data_model = args[1]
        data = None
        if len(args) >= 3:
            data = args[2]
        super().__init__(name, data_model, data)
        
        self.activity_column = None
        self.timestamp_column = None
        self.case_column = None
                
        self.caseTable = None
        if len(args) <= 3:
            activity_table = None
            for table in data_model.process_configurations:
                if table.activity_table.name == name:
                    activity_table = table
                    break
            if activity_table:
                self.activity_column = activity_table.activity_column
                self.timestamp_column = activity_table.timestamp_column
                self.case_column = activity_table.case_column
                self.caseTable = activity_table.case_table.name
        elif len(args) == 6:
            # maybe create case table and set foreign key relation (can do this since we have data)
            self.activity_column = args[3]
            self.case_column = args[4]
            self.timestamp_column = args[5]

In [16]:
class Tables:
    def __init__(self):
        self.collection = []
    
    def addTable(self, table):
        self.collection.append(table)
    
    def removeTable(self, table):
        self.collection.remove(table)

    def removeByName(self, name):
        for i in self.collection:
            if i.name == name:
                self.collection.remove(i)
      
    def output(self):
        return [table.name for table in self.collection]

    # returns first occurence
    def find(self, name):
        for table in self.collection:
            if table.name == name:
                return table

In [17]:
class DataModel:

    def __init__(self, data_model):
        self.data_model = data_model
        
        self.tables = Tables()
        self.activity_tables = Tables()
        
        print("Fetching process configurations...")
        for table in data_model.process_configurations:
            self.activity_tables.addTable(ActivityTable(table.activity_table.name, data_model))

        print("Creating tables...")
        for table in data_model.tables:
            act = self.activity_tables.find(table.name)
            if act is not None:
                self.tables.addTable(act)
            else:        
                self.tables.addTable(Table(table.name, data_model))
        
        print("Fetching foreign keys...")
        self.foreignKeys = data_model.foreign_keys
        
        print("Creating foreign key graph...")
        self.foreignKeyGraph = None

        self.updateForeignKeyGraph()
        
        self.objectRelationships = {}
        
        # add other optimization attributes: 
            # all calculation methods should save result to avoid duplicate calculations
        
    def updateForeignKeyGraph(self):
        graph = {}
        for table in self.tables.collection:
            connected_nodes = []
            for dictionary in table.foreignKeys["as_source"]:
                connected_nodes.append(dictionary["target_table"])
            for dictionary in table.foreignKeys["as_target"]:
                connected_nodes.append(dictionary["source_table"])
            graph[table.name] = connected_nodes

        G = nx.Graph()
        for i in graph:
            G.add_node(i)
            for j in graph[i]:
                G.add_edge(i, j)
        
        self.foreignKeyGraph = G
    
        
    def calcForeignKeyPath(self, table1, table2):
        try: 
            return nx.algorithms.shortest_paths.generic.shortest_path(self.foreignKeyGraph, source=table1, target=table2)
        except:
            print("No path found")
            return []

            
    # mergePath format: [{"leftTable": table1, "leftColumn": column1, "rightTable": table2, "rightColumn": column2}, {"leftTable": table3, "leftColumn": column3, "rightTable": table4, "rightColumn": column4}]
    def calcMergePath(self, table1, table2):
        path = self.calcForeignKeyPath(table1, table2)
        
        if len(path) == 0:
            return
        
        potential_relations2 = self.tables.find(path[0]).foreignKeys["as_source"] + self.tables.find(path[0]).foreignKeys["as_target"]

        mergePath = []
        for i in range(len(path)-1):
            potential_relations1 = potential_relations2
            potential_relations2 = self.tables.find(path[i+1]).foreignKeys["as_source"] + self.tables.find(path[i+1]).foreignKeys["as_target"]
            key_relation = {}
            for relation in potential_relations1:
                if relation in potential_relations2:
                    key_relation = relation
                    break
            if key_relation == {}:
                print("No Relation Found!")
                return mergePath
            mergePath.append({"leftTable": key_relation["source_table"], "leftColumn": key_relation["columns"][0][0], 
                              "rightTable": key_relation["target_table"], "rightColumn": key_relation["columns"][0][1]})
        return mergePath
    
    
    def calcObjectRelationships(self, activityTable1, activityTable2):
        if (activityTable1, activityTable2) in self.objectRelationships:
            return self.objectRelationships[(activityTable1, activityTable2)]        
        
        actTable1 = self.activity_tables.find(activityTable1)
        actTable2 = self.activity_tables.find(activityTable2)
        
        if actTable1 is None or actTable2 is None:
            print("Not activity table")
            return
        
        mergePath = self.calcMergePath(activityTable1, activityTable2)

        df = actTable1.getData()
        for relation in mergePath:
            df = df.merge(self.tables.find(relation["leftTable"]).getData() \
                .merge(self.tables.find(relation["rightTable"]).getData(), \
                       left_on=relation["leftColumn"], right_on=relation["rightColumn"]))
        
        columns_to_keep = [actTable1.case_column, actTable2.case_column]
        
        df = df[list(set(columns_to_keep).intersection(df.columns))]
        
        df.drop_duplicates(inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        self.objectRelationships[(activityTable1, activityTable2)] = df
        # save table as actual Table() object including foreign keys to activity tables...
        return df

    
    def groupEvents(self, activityTableName):
        # group events in table with same timestamp + activity name
        table = self.activity_tables.find(activityTableName)
        if table is None:
            print(activityTableName + " is not activity table")
            return
        
        grouped = self.tables.find(activityTableName + "_GROUPED")
        if grouped is not None:
            return grouped
        
        events = {}

        for ev in table.getData().to_dict("records"):
            evv = (ev[table.activity_column], ev[table.timestamp_column])
            if evv not in events:
                events[evv] = frozenset()
            if isinstance(ev[table.case_column],frozenset):
                events[evv] = events[evv].union(ev[table.case_column])
            else:
                events[evv] = events[evv].union(frozenset([ev[table.case_column]]))

        df = []
        for key, val in events.items():
            df.append( {"OBJECTS": val, "ACTIVITY": key[0], "TIMESTAMP": key[1]} )
        
        # adding to tables/activityTables collection
        actTable = ActivityTable(activityTableName + "_GROUPED", self.data_model, 
                                 pd.DataFrame(df), "ACTIVITY", "OBJECTS", "TIMESTAMP")
        self.tables.addTable(actTable)
        self.activity_tables.addTable(actTable)
        
        
        ################ START OUTSOURCE
        
        
        # create connection tables
        grouped = actTable

        unique = []
        for obj in table.getData()[table.case_column]:
            if obj not in unique:
                unique.append(obj)

        values = []
        for i in range(len(unique)):
            for j in grouped.getData()[grouped.case_column]:
                if set([unique[i]]).intersection(set(j)) != set():
                    values.append((j, unique[i]))
        data = pd.DataFrame(values, columns=[grouped.name + "_COLUMN", table.name + "_COLUMN"])
        data.drop_duplicates(inplace=True)
        data.reset_index(drop=True, inplace=True)
            
        newTable = Table("CONNECT_" + grouped.name + "_AND_" + table.name, self.data_model, data)
            
        # add foreign keys
        newTable.foreignKeys["as_target"].append({'source_table': grouped.name,
                                                       'target_table': newTable.name,
                                                       'columns': [(grouped.case_column, grouped.name + "_COLUMN")]})
        grouped.foreignKeys["as_source"].append({'source_table': grouped.name,
                                                       'target_table': newTable.name,
                                                       'columns': [(grouped.case_column, grouped.name + "_COLUMN")]})
        newTable.foreignKeys["as_source"].append({'source_table': newTable.name,
                                                       'target_table': table.name,
                                                       'columns': [(table.name + "_COLUMN", table.case_column)]})
        table.foreignKeys["as_target"].append({'source_table': newTable.name,
                                                       'target_table': table.name,
                                                       'columns': [(table.name + "_COLUMN", table.case_column)]})
        self.tables.addTable(newTable)
        
        self.updateForeignKeyGraph()
        
        
        ############### END OUTSOURCE
        
        
        
        # also group other attributes...
        return actTable
        


    def union(self, actTableName1, actTableName2):
        # use objectRelationships function to calculate intermediate table
        table1 = self.activity_tables.find(actTableName1)
        table2 = self.activity_tables.find(actTableName2)
        if table1 is None or table2 is None:
            print("Non activity table passed")
            return
        
        union = self.tables.find(actTableName1 + "_" + actTableName2 + "_UNION")
        if union is not None:
            return union
        
        relationship = self.calcObjectRelationships(actTableName1, actTableName2)

        df = table1.getData().merge(relationship, on=table1.case_column) \
                            .merge(table2.getData(), 
                                   left_on=[table2.case_column, table1.activity_column, table1.timestamp_column], 
                                   right_on=[table2.case_column, table2.activity_column, table2.timestamp_column])

        
        newDf2 = table2.getData()[[table2.case_column, table2.activity_column, table2.timestamp_column]]
        newDf2.columns = ["OBJECTS", "ACTIVITY", "TIMESTAMP"]
        
        group = {}
        
        for row in df.to_dict("records"):
            key = (row[table1.case_column], row[table1.activity_column], row[table1.timestamp_column])
            if key not in group:
                group[key] = frozenset()
            if isinstance(row[table2.case_column],frozenset):
                group[key] = group[key].union(row[table2.case_column])
            else:
                group[key] = group[key].union([row[table2.case_column]])
            i = newDf2[((newDf2.OBJECTS == row[table2.case_column]) &( newDf2.TIMESTAMP == key[2]) & (newDf2.ACTIVITY == key[1]))].index
            newDf2.drop(i, inplace=True)
            
        newDf = table1.getData()[[table1.case_column, table1.activity_column, table1.timestamp_column]]
        newDf.columns = ["OBJECTS", "ACTIVITY", "TIMESTAMP"]
        
        newDf = newDf.append(newDf2).reset_index(drop=True)
        
        
        for index, row in newDf.iterrows():
            key = (row["OBJECTS"], row["ACTIVITY"],row["TIMESTAMP"])
            if key in group:
                if isinstance(newDf.at[index, "OBJECTS"], frozenset):
                    newDf.at[index, "OBJECTS"] = newDf.at[index, "OBJECTS"].union(group[key])
                else:
                    newDf.at[index, "OBJECTS"] = frozenset([newDf.at[index, "OBJECTS"]]).union(group[key])
            else:
                # if no new objects will be added
                if not isinstance(newDf.at[index, "OBJECTS"], frozenset):
                    newDf.at[index, "OBJECTS"] = frozenset([newDf.at[index, "OBJECTS"]])

        newDf = newDf.sort_values(by='TIMESTAMP').reset_index(drop=True)
        
        act = ActivityTable(actTableName1 + "_" + actTableName2 + "_UNION", self.data_model, newDf, "ACTIVITY", "OBJECTS", "TIMESTAMP")
        self.tables.addTable(act)
        self.activity_tables.addTable(act)
        
        # create connection tables
        for table in [table1, table2]:
            unioned = act

            unique = []
            for obj in table.getData()[table.case_column]:
                if obj not in unique:
                    unique.append(obj)

            values = []
            for i in range(len(unique)):
                for j in unioned.getData()[unioned.case_column]:
                    if set([unique[i]]).intersection(set(j)) != set():
                        values.append((j, unique[i]))
            data = pd.DataFrame(values, columns=[unioned.name + "_COLUMN", table.name + "_COLUMN"])
            data.drop_duplicates(inplace=True)
            data.reset_index(drop=True, inplace=True)
            
            newTable = Table("CONNECT_" + unioned.name + "_AND_" + table.name, self.data_model, data)
            
            # add foreign keys
            newTable.foreignKeys["as_target"].append({'source_table': unioned.name,
                                                       'target_table': newTable.name,
                                                       'columns': [(unioned.case_column, unioned.name + "_COLUMN")]})
            unioned.foreignKeys["as_source"].append({'source_table': unioned.name,
                                                       'target_table': newTable.name,
                                                       'columns': [(unioned.case_column, unioned.name + "_COLUMN")]})
            newTable.foreignKeys["as_source"].append({'source_table': newTable.name,
                                                       'target_table': table.name,
                                                       'columns': [(table.name + "_COLUMN", table.case_column)]})
            table.foreignKeys["as_target"].append({'source_table': newTable.name,
                                                       'target_table': table.name,
                                                       'columns': [(table.name + "_COLUMN", table.case_column)]})
            self.tables.addTable(newTable)
        
        self.updateForeignKeyGraph()
        
        # what about other attributes?
        
        return act

In [18]:
dm = DataModel(data_model)

Fetching process configurations...
Creating tables...
Fetching foreign keys...
Creating foreign key graph...


In [19]:
unioned = dm.union("order_EVENTS", "item_EVENTS")

2022-01-19 16:12:54 - pycelonis: PQL Export started...
2022-01-19 16:12:55 - pycelonis: PQL Export status: DONE
2022-01-19 16:13:04 - pycelonis: PQL Export started...
2022-01-19 16:13:04 - pycelonis: PQL Export status: DONE
2022-01-19 16:13:16 - pycelonis: PQL Export started...
2022-01-19 16:13:16 - pycelonis: PQL Export status: DONE
2022-01-19 16:13:28 - pycelonis: PQL Export started...
2022-01-19 16:13:28 - pycelonis: PQL Export status: DONE
2022-01-19 16:13:39 - pycelonis: PQL Export started...
2022-01-19 16:13:39 - pycelonis: PQL Export status: DONE
2022-01-19 16:13:46 - pycelonis: PQL Export started...
2022-01-19 16:13:46 - pycelonis: PQL Export status: DONE
2022-01-19 16:13:54 - pycelonis: PQL Export started...
2022-01-19 16:13:54 - pycelonis: PQL Export status: DONE


/usr/lib/python3/dist-packages/pandas/core/frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
unioned.getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, order1, item1)",place order,2020-07-09 08:20:00
1,(order2),place order,2020-07-09 08:20:00
2,(item1),check availability,2020-07-09 08:21:00
3,(item1),confirm availability,2020-07-09 08:23:00
4,(order1),confirm order,2020-07-09 08:30:00
5,(order1),end order,2020-07-09 08:35:00


In [21]:
dm.calcObjectRelationships(unioned.name, "item_EVENTS")

,OBJECTS,CASE_item
0,"(item2, order1, item1)",item1
1,"(item2, order1, item1)",item2
2,(item1),item1


In [22]:
dm.union(unioned.name, "order_EVENTS").getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, order1, item1)",place order,2020-07-09 08:20:00
1,(order2),place order,2020-07-09 08:20:00
2,(item1),check availability,2020-07-09 08:21:00
3,(item1),confirm availability,2020-07-09 08:23:00
4,(order1),confirm order,2020-07-09 08:30:00
5,(order1),end order,2020-07-09 08:35:00


In [23]:
allUnion = dm.union(unioned.name, "package_EVENTS")
allUnion.getData()

2022-01-19 16:14:27 - pycelonis: PQL Export started...
2022-01-19 16:14:27 - pycelonis: PQL Export status: DONE


,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, order1, item1, package1)",place order,2020-07-09 08:20:00
1,(order2),place order,2020-07-09 08:20:00
2,(item1),check availability,2020-07-09 08:21:00
3,(item1),confirm availability,2020-07-09 08:23:00
4,(order1),confirm order,2020-07-09 08:30:00
5,(package1),send package,2020-07-09 08:31:00
6,(order1),end order,2020-07-09 08:35:00


In [24]:
dm.groupEvents(allUnion.name).getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, order1, item1, order2, package1)",place order,2020-07-09 08:20:00
1,(item1),check availability,2020-07-09 08:21:00
2,(item1),confirm availability,2020-07-09 08:23:00
3,(order1),confirm order,2020-07-09 08:30:00
4,(package1),send package,2020-07-09 08:31:00
5,(order1),end order,2020-07-09 08:35:00


In [25]:
dm.groupEvents("item_EVENTS").getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, item1)",place order,2020-07-09 08:20:00
1,(item1),check availability,2020-07-09 08:21:00
2,(item1),confirm availability,2020-07-09 08:23:00


In [26]:
dm.union(dm.groupEvents("order_EVENTS").name, dm.groupEvents("item_EVENTS").name).getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(order2, order1)",place order,2020-07-09 08:20:00
1,"(item2, item1)",place order,2020-07-09 08:20:00
2,(item1),check availability,2020-07-09 08:21:00
3,(item1),confirm availability,2020-07-09 08:23:00
4,(order1),confirm order,2020-07-09 08:30:00
5,(order1),end order,2020-07-09 08:35:00


In [27]:
dm.groupEvents(dm.union("order_EVENTS", "item_EVENTS").name).getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, order2, order1, item1)",place order,2020-07-09 08:20:00
1,(item1),check availability,2020-07-09 08:21:00
2,(item1),confirm availability,2020-07-09 08:23:00
3,(order1),confirm order,2020-07-09 08:30:00
4,(order1),end order,2020-07-09 08:35:00


In [28]:
dm.union(dm.union(unioned.name, "order_EVENTS").name, "order_EVENTS").getData()

,OBJECTS,ACTIVITY,TIMESTAMP
0,"(item2, order1, item1)",place order,2020-07-09 08:20:00
1,(order2),place order,2020-07-09 08:20:00
2,(item1),check availability,2020-07-09 08:21:00
3,(item1),confirm availability,2020-07-09 08:23:00
4,(order1),confirm order,2020-07-09 08:30:00
5,(order1),end order,2020-07-09 08:35:00


# TODO:
### Add comments, better variable names...
### other attributes
### groupEvents should also create new activityTable + foreign keys...
### outsource creating new table with foreignKeys ... to new method
### for sections where we turn variable to set, use one line statements ... x i a else y...
### table relationships??? should after union, there be relatoinship (order1, order1) in connector table?
### problems when 'stacking' union tables with already unioned table (sometimes objects are set, in beginning not...)
### better names for new tables created (GROUPED, UNION, CONNECT...)
### if groupEvents applied first and then union, objects aren't grouped properly?